# SCRAPING AMAZON BOOK DATA 

## Importing Useful Libraries

In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs

## Getting Genre data

In [2]:
r = requests.get("https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_unv_books_1_1318158031_2")
# Convert to a beautiful soup object
soup = bs(r.content)
# Print out the HTML
contents = soup.prettify()


- Getting genre name and their website URL's

In [3]:
info_box = soup.find(class_='_p13n-zg-nav-tree-all_style_zg-browse-group__88fbz')
info_rows = info_box.find_all("a")
info_dict ={}
base_path = 'https://www.amazon.in'
for row in info_rows:
    relative_path = row['href']
    full_path= base_path + relative_path 
    title = list(row)[0]
    info_dict[title] = full_path 

### Getting Adventure Genre Book Info

In [4]:
r = requests.get("https://www.amazon.in/gp/bestsellers/books/1318158031")

# Convert to a beautiful soup object
soupb = bs(r.content)

# Print out the HTML
contents = soupb.prettify()


In [5]:
book_info = soupb.find_all(class_='_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y')
book_dict = {}
book_name=[]
author = []
for index,row in enumerate(book_info):
    title = list(row)[0]
    if index==0 or index%2==0:
        book_name.append(title)
    else:
        author.append(title)     

-- Finding ratings and remaining data according to our structure

In [6]:
#ratings
ratings = soupb.find_all(class_ = 'a-icon-alt')
book_rating = []
for row in ratings:
    rating = list(row)[0].split()[0]
    book_rating.append(float(rating))

In [7]:
#no of reviews
reviews = soupb.find_all(class_ = "a-size-small")
book_no_of_reviews= []
text_list=[]
for index,row in enumerate(reviews):
    rating = list(row)[0].text
    text_list.append(rating)

i=2
for index,element in enumerate(text_list):
    if index==i:
        book_no_of_reviews.append(element)
        i=i+5
    
book_no_of_reviews = book_no_of_reviews[0:50]


In [8]:
#publish_type
publish_type = soupb.find_all(class_ = "a-size-small a-color-secondary a-text-normal")
book_publish_type= []
for row in publish_type:
    publish = list(row)[-1]
    book_publish_type.append(publish)

In [9]:
#price
price = soupb.find_all(class_ = "p13n-sc-price")
price_list=[]
for row in price:
    price = list(row)[0]
    price_list.append(price)

In [10]:
df = pd.DataFrame(list(zip(book_name, author,book_rating,book_no_of_reviews,book_publish_type,price_list)),
               columns =['book_name', 'author','book_rating','book_no_of_reviews','book_publish_type','price'])
df.head(5)

,book_name,author,book_rating,book_no_of_reviews,book_publish_type,price
0,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,"55,326",Kindle Edition,₹284.05
1,War of Lanka (Ram Chandra Series Book 4),Amish Tripathi,4.3,"2,301",Paperback,₹352.00
2,The Complete Novels of Sherlock Holmes,Arthur Conan Doyle,4.5,"15,471",Paperback,₹139.00
3,"Samsara: Enter the Valley of the Gods (""India'...",Saksham Garg,4.5,326,Paperback,₹150.00
4,Maalai Pozhuthin Mayakkathile (Tamil Edition),Sashi Murali,4.2,55,Kindle Edition,₹409.00


### Now Making Function for extracting book info of all genre

In [11]:
result = []
df_dict ={}
def book_info():
    for itm in info_dict.items():
        r = requests.get(itm[1])
        # Convert to a beautiful soup object
        soup = bs(r.content)
        # Print out the HTML
        contents = soup.prettify()

        book_info = soup.find_all(class_='_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y')
        book_dict = {}
        book_name=[]
        author = []
        for index,row in enumerate(book_info):
            title = list(row)[0]
            if index==0 or index%2==0:
                book_name.append(title)
            else:
                author.append(title)

        #ratings
        ratings = soup.find_all(class_ = 'a-icon-alt')
        book_rating = []
        for row in ratings:
            rating = list(row)[0].split()[0]
            book_rating.append(float(rating))

        #no of reviews
        reviews = soup.find_all(class_ = "a-size-small")
        book_no_of_reviews= []
        text_list=[]
        for index,row in enumerate(reviews):
            rating = list(row)[0].text
            text_list.append(rating)
        i=2
        for index,element in enumerate(text_list):
            if index==i:
                book_no_of_reviews.append(element)
                i=i+5
        book_no_of_reviews = book_no_of_reviews[0:len(author)]

        #publish_type
        publish_type = soup.find_all(class_ = "a-size-small a-color-secondary a-text-normal")
        book_publish_type= []
        for row in publish_type:
            publish = list(row)[-1]
            book_publish_type.append(publish)

        #price
        price = soup.find_all(class_ = "p13n-sc-price")
        price_list=[]
        for row in price:
            price = list(row)[0]
            price_list.append(price)

        
        genre_list=[]
        for j in range(len(author)):
            genre_list.append(itm[0])
        
        df = pd.DataFrame(list(zip(genre_list,book_name, author,book_rating,book_no_of_reviews,book_publish_type,price_list)),
                   columns =['Genre','book_name', 'author','book_rating','book_no_of_reviews','book_publish_type','price'])
   
        df_dict[itm[0]] = df
    
    # Joining all dataframes
   
    for i in df_dict.values():
        result.append(i)
    global final_result
    final_result=pd.concat(result)
    
    return final_result

    

## Some Data Exploration and Cleaning

In [12]:
book_info()


,Genre,book_name,author,book_rating,book_no_of_reviews,book_publish_type,price
0,Action & Adventure,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,"55,326",Kindle Edition,₹284.05
1,Action & Adventure,War of Lanka (Ram Chandra Series Book 4),Amish Tripathi,4.3,"2,301",Paperback,₹352.00
2,Action & Adventure,The Complete Novels of Sherlock Holmes,Arthur Conan Doyle,4.5,"15,471",Paperback,₹139.00
3,Action & Adventure,"Samsara: Enter the Valley of the Gods (""India'...",Saksham Garg,4.5,326,Paperback,₹150.00
4,Action & Adventure,Maalai Pozhuthin Mayakkathile (Tamil Edition),Sashi Murali,4.2,55,Kindle Edition,₹409.00
...,...,...,...,...,...,...,...
44,Travel,Ernesto Che Guevara,"A SAINT, A FOLK TALE AND OTHER STORIES: LESSER...",4.6,Paperback,Paperback,₹259.00
45,Travel,Rana Safvi,French: Learn French For Beginners Including F...,4.1,Paperback,Paperback,₹191.00
46,Travel,Language Learning University,India: A Wounded Civilization,4.4,Kindle Edition,Kindle Edition,₹420.00
47,Travel,Sir V. S. Naipaul,The Three Musketeers : Illustrated Abridged Ch...,4.3,Paperback,Paperback,₹324.00


In [13]:
final_result['Genre'].value_counts()

Action & Adventure                       50
Sports                                   50
Comics & Mangas                          50
Reference                                50
Humour                                   50
Health, Fitness & Nutrition              49
Society & Social Sciences                49
Science & Mathematics                    49
School Books                             49
Historical Fiction                       49
History                                  49
Travel                                   49
Fantasy, Horror & Science Fiction        49
Biographies, Diaries & True Accounts     49
Children's & Young Adult                 49
Engineering                              49
Computing, Internet & Digital Media      49
Crime, Thriller & Mystery                49
Sciences, Technology & Medicine          48
Crafts, Home & Lifestyle                 48
Religion                                 48
Politics                                 48
Language, Linguistics & Writing 

In [14]:
final_result.tail(50)

,Genre,book_name,author,book_rating,book_no_of_reviews,book_publish_type,price
49,Sports,THE MASTER: THE BRILLIANT CAREER OF ROGER FEDERER,Christopher Clarey,4.5,641,Paperback,₹591.00
0,Travel,Twenty-One Nights in Paris: Escape to Paris wi...,Leonie Mack,4.4,223,Kindle Edition,₹109.74
1,Travel,Masala Lab: The Science of Indian Cooking: The...,Krish Ashok,4.5,"1,958",Paperback,₹259.00
2,Travel,Murder in Tuscany: The start of a BRAND NEW co...,T A Williams,4.4,466,Kindle Edition,₹85.74
3,Travel,India Map (Laminated Both Sides ) - With New U...,Dreamland Publications,4.4,"5,761",Poster,₹129.00
4,Travel,World Map - Laminated Both Sides,Dreamland Publications,4.3,"9,784",Wall Chart,₹129.00
5,Travel,Where the Sun Never Sets (Signed by the author),Stuti Changle,4.2,177,Paperback,₹162.00
6,Travel,I Wish I Could Tell Her (Order now to get a si...,Ajay K Pandey,4.5,321,Paperback,₹140.00
7,Travel,Normal People: One million copies sold,Sally Rooney,4.2,"48,002",Paperback,₹323.00
8,Travel,Zen: The Art of Simple Living,Shunmyo Masuno,4.6,"2,431",Hardcover,₹379.00


In [15]:
final_result['book_no_of_reviews']


0             55,326
1              2,301
2             15,471
3                326
4                 55
           ...      
44         Paperback
45         Paperback
46    Kindle Edition
47         Paperback
48         Hardcover
Name: book_no_of_reviews, Length: 1501, dtype: object

In [16]:
final_result['price'].unique()

array(['₹284.05', '₹352.00', '₹139.00', '₹150.00', '₹409.00', '₹160.00',
       '₹235.00', '₹199.00', '₹836.00', '₹229.00', '₹190.00', '₹279.30',
       '₹149.00', '₹100.00', '₹627.00', '₹123.00', '₹213.00', '₹209.00',
       '₹85.74', '₹313.95', '₹1,003.00', '₹223.00', '₹154.46', '₹238.00',
       '₹259.00', '₹275.50', '₹407.00', '₹498.00', '₹162.00', '₹316.00',
       '₹1,101.00', '₹452.00', '₹599.00', '₹449.00', '₹180.00', '₹629.00',
       '₹99.00', '₹109.74', '₹496.00', '₹101.00', '₹1,025.00', '₹330.00',
       '₹621.00', '₹200.00', '₹175.00', '₹126.00', '₹89.00', '₹349.00',
       '₹155.00', '₹400.00', '₹135.00', '₹140.00', '₹389.00', '₹451.00',
       '₹53.00', '₹170.00', '₹454.00', '₹324.00', '₹350.00', '₹159.00',
       '₹112.00', '₹194.00', '₹480.00', '₹369.00', '₹95.00', '₹501.00',
       '₹265.00', '₹531.00', '₹584.00', '₹739.00', '₹359.00', '₹404.00',
       '₹299.00', '₹210.00', '₹185.00', '₹191.00', '₹52.00', '₹1,629.00',
       '₹967.77', '₹361.34', '₹349.60', '₹559.00'

In [17]:
final_result['book_no_of_reviews'].unique()

array(['55,326', '2,301', '15,471', '326', '55', '874', '136,706', '136',
       '12,673', '191', '2,296', '73', '438', '12,128', '11,515', '7,445',
       '19', '466', '30,073', '37,482', '8,349', '35,844', '32,399',
       '633', '1,617', '10,515', '42,485', '4', '8,233', '34,483',
       '19,070', '177', '3,923', '61,698', '6,094', '16,785', '58', '150',
       '7,806', '201', '3,724', '216', '87,787', '1,064', '24,319',
       '23,702', '1,314', '480', '7,098', '2,270', '5,968', '409', '41',
       '17', '1,397', '2,537', '4,062', '37', '1,628', '1,323', '27,723',
       '93', '343', '3,486', '1,979', 'Spiral-bound', 'Paperback',
       'Hardcover', 'Audible Audiobook', 'Sheet music',
       'Roshan Kumar  Chaudhary ', 'ROHIT VAIDWAN', 'Tom', 'NCERT',
       'Rays Of Ink', 'Anuj Tiwari', '270', '306', '65', '4,310', '945',
       '1', '255', '50,792', '60,048', '14,777', '81,892', '10,044',
       '5,816', '924', '8,305', '7,756', '16,891', '1,712', '5,314',
       '8,679', '40', '

In [18]:
# Converting Book_no_of_reviews into integers
# handle such cases : x=['2300','2,000','abc']
def converting_to_int(i):
    if len(i.split(','))==1:
        try:
            i = int(i)
            return i
        except:
            i=0
            return i
    else:
        try:
            if len(i.split(','))==2:
                i = int(i.split(',')[0])*1000 + int(i.split(',')[1])
                return i
        except:
            i=0
            return i
            

In [19]:
final_result['book_no_of_reviews'] = final_result['book_no_of_reviews'].apply(converting_to_int)


In [20]:
final_result['book_no_of_reviews'].value_counts()

0        813
2301      11
1          9
5          6
28         5
        ... 
1072       1
2070       1
24054      1
49058      1
2114       1
Name: book_no_of_reviews, Length: 448, dtype: int64

In [21]:
final_result = final_result[final_result.book_no_of_reviews != 0]

In [22]:
final_result


,Genre,book_name,author,book_rating,book_no_of_reviews,book_publish_type,price
0,Action & Adventure,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,55326,Kindle Edition,₹284.05
1,Action & Adventure,War of Lanka (Ram Chandra Series Book 4),Amish Tripathi,4.3,2301,Paperback,₹352.00
2,Action & Adventure,The Complete Novels of Sherlock Holmes,Arthur Conan Doyle,4.5,15471,Paperback,₹139.00
3,Action & Adventure,"Samsara: Enter the Valley of the Gods (""India'...",Saksham Garg,4.5,326,Paperback,₹150.00
4,Action & Adventure,Maalai Pozhuthin Mayakkathile (Tamil Edition),Sashi Murali,4.2,55,Kindle Edition,₹409.00
...,...,...,...,...,...,...,...
38,Travel,Into Thin Air: A Personal Account of the Evere...,Jon Krakauer,4.6,8557,Paperback,₹298.00
39,Travel,"Our World in Pictures: Countries, Cultur: A Vi...",DK,4.8,992,Hardcover,₹583.00
40,Travel,A Harvest Murder: The BRAND NEW cozy crime mur...,Frances Evesham,4.4,1272,Kindle Edition,₹283.00
41,Travel,A Year at the French Farmhouse: Escape to Fran...,Gillian Harvey,4.0,1473,Kindle Edition,₹204.27


In [23]:
#converting price into int $452.00 , $2,088.00
#x='$555'
#x = x[1:]
#print(x)

#y='2,300.00'
#y.split(',')

In [24]:
def converting_price_int(i):
    i = i[1:]
    if len(i.split(','))==2:
        try:
            i = float(i.split(',')[0])*1000 + float(i.split(',')[1])
            return i
        except:
            return float(i[1:])
    else:
        i= float(i.split('.')[0]) + float(i.split('.')[1])/100
        return i


In [25]:
final_result['price'] = final_result['price'].apply(converting_price_int)


C:\Users\OSAMA SHAMIM\AppData\Local\Temp\ipykernel_17152\2473494458.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result['price'] = final_result['price'].apply(converting_price_int)


In [26]:
final_result

,Genre,book_name,author,book_rating,book_no_of_reviews,book_publish_type,price
0,Action & Adventure,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,55326,Kindle Edition,284.05
1,Action & Adventure,War of Lanka (Ram Chandra Series Book 4),Amish Tripathi,4.3,2301,Paperback,352.00
2,Action & Adventure,The Complete Novels of Sherlock Holmes,Arthur Conan Doyle,4.5,15471,Paperback,139.00
3,Action & Adventure,"Samsara: Enter the Valley of the Gods (""India'...",Saksham Garg,4.5,326,Paperback,150.00
4,Action & Adventure,Maalai Pozhuthin Mayakkathile (Tamil Edition),Sashi Murali,4.2,55,Kindle Edition,409.00
...,...,...,...,...,...,...,...
38,Travel,Into Thin Air: A Personal Account of the Evere...,Jon Krakauer,4.6,8557,Paperback,298.00
39,Travel,"Our World in Pictures: Countries, Cultur: A Vi...",DK,4.8,992,Hardcover,583.00
40,Travel,A Harvest Murder: The BRAND NEW cozy crime mur...,Frances Evesham,4.4,1272,Kindle Edition,283.00
41,Travel,A Year at the French Farmhouse: Escape to Fran...,Gillian Harvey,4.0,1473,Kindle Edition,204.27


In [27]:
final_result.to_csv(index=True)

',Genre,book_name,author,book_rating,book_no_of_reviews,book_publish_type,price\r\n0,Action & Adventure,Harry Potter and the Philosopher\'s Stone,J.K. Rowling,4.7,55326,Kindle Edition,284.05\r\n1,Action & Adventure,War of Lanka (Ram Chandra Series Book 4),Amish Tripathi,4.3,2301,Paperback,352.0\r\n2,Action & Adventure,The Complete Novels of Sherlock Holmes,Arthur Conan Doyle,4.5,15471,Paperback,139.0\r\n3,Action & Adventure,"Samsara: Enter the Valley of the Gods (""India\'s answer to Harry Potter"")",Saksham Garg,4.5,326,Paperback,150.0\r\n4,Action & Adventure,Maalai Pozhuthin Mayakkathile (Tamil Edition),Sashi Murali,4.2,55,Kindle Edition,409.0\r\n5,Action & Adventure,The Hidden Hindu,Akshat Gupta,4.3,874,Paperback,160.0\r\n6,Action & Adventure,THE SILENT PATIENT,Alex Michaelides,4.5,136706,Paperback,235.0\r\n7,Action & Adventure,தீயில் தென்றலாய் உறவிதுவோ: theeyil thendralaai uravithuvo (Tamil Edition),மேக வாணி Megavani,4.5,136,Kindle Edition,199.0\r\n8,Action & Adventure,The Immortal